# Lecture 12, Methods for multiobjective optimization 

Methods for multiobjective optimization are often charactermized by the involvement of the decision maker in the process.

The types of methods are
* no preference methods, where the decision maker does not play a role,
* a priori methods, where the decision maker gives his/her preference information at first and then the optimization method find the best match to that preference information,
* a posteriori methods, where the optimization methods try to characterize all/find a good represenatation of the Pareto optimal solutions and the decision maker chooses the most preferred one of those,
* interactive methods, where the optimization method and the decision maker alternate in iteratively search for the most preferred solution.

## Notation

For short, let us denote the feasible set $\{x\in\mathbb R^n: g_j(x) \geq 0 \text{ for all }j=1,\ldots,J \text{ and } h_k(x) = 0\text{ for all }k=1,\ldots,K\}$ by $S$.

## Problem to be studied

We study a hypothetical decision problem of buying a car, when you can choose to have a car with power between (denoted by $p$) 50 and 200 kW and average consumption (denoted by $c$) per 100 km between 3 and 10 l. However, in addition to the average consumption and power, you need to decide the volume of the cylinders (v), which may be between 1000 $cm^3$ and $4000cm^3$. Finally, the price of the car follows now a function 
$$
\left(\sqrt{\frac{p-50}{50}}+\frac{p-50}{50}^2+\frac{10-c}{10} +10^{-5}\left(v-\left(1000+3000\frac{p-50}{150}\right)\right)^2\right)10000+5000
$$
in euros. This problem can be formulated as a multiobjective optimization problem
$$
\begin{align}
\min \quad & \{c,-p,P\},\\
\text{s.t. }\quad
&50\leq p\leq 200\\
&3\leq c\leq 10\\
&1000\leq v\leq 4000,\\
\text{where }\quad&P = \left(\sqrt{\frac{p-50}{50}}+\frac{p-50}{50}^2+\frac{10-c}{10} + \frac{p-50}{50}\frac{10-c}{10}\right)10000+5000
\end{align}
$$

In [113]:
#Let us define a Python function which returns the value of this
import math
def car_problem(c,p,v):
#    import pdb; pdb.set_trace()
    return [#Objective function values
        c,-p,
        (math.sqrt((p-40.)/50.)+((p-50.)/50.)**2+
        (10.-c)/10.+1e-5*(v-(1000.+3000.*(p-50.)/150.))**2)*10000.+5000.] 

In [114]:
car_problem(3,50,1000)

[3, -50, 16472.135954999576]

## No preference methods

* Usually only for situations, where the decision maker is not available or does not want to get involved
* These just compute a single Pareto optimal solution, which is in somehow mathematically thought as the best compromise

### Method of Global criterion

Involved minimization of the p-norm of $f(x)-z^{ideal}$, that is we solve the problem
$$
\min_{x\in S} \|f(x) - z^{ideal}\|_p.
$$

![alt text](images/mgc.svg "Method of global criterion")


### Applying to our problem studied


In [115]:
#Calculating the ideal
from scipy.optimize import minimize
import ad
def calc_ideal(f):
    ideal = [0]*3 #Because three objectives
    bounds = ((3,10),(50,200),(1000,4000)) #Bounds of the problem
    for i in range(3):
        res=minimize(
            #Minimize each objective at the time
            lambda x: f(x[0],x[1],x[2])[i], [3,50,1000], method='SLSQP'
            #Jacobian using automatic differentiation
            ,jac=ad.gh(lambda x: f(x[0],x[1],x[2])[i])[0]
            #bounds given above
            ,bounds = bounds,options = {'disp':True})
        ideal[i]=res.fun
    return ideal

In [116]:
ideal = calc_ideal(car_problem)
print ideal

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3.0
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -200.0
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9472.13571906
            Iterations: 3
            Function evaluations: 3
            Gradient evaluations: 2
[3.0, -200.0, 9472.1357190646031]


In [117]:
import numpy as np
def global_criterion_method(f,p):
    ideal = calc_ideal(f)
    bounds = ((3,10),(50,200),(1000,4000)) #Bounds of the problem
    obj = lambda x: np.linalg.norm(np.array(f(x[0],x[1],x[2]))-np.array(ideal),ord=p)
    res=minimize(
            #Minimize p distance from the ideal
            obj, [3,50,1000], method='SLSQP'
            #Jacobian using automatic differentiation
            ,jac=ad.gh(obj)[0]
            #bounds given above
            ,bounds = bounds,options = {'disp':True})
    return ideal

In [118]:
ideal = global_criterion_method(car_problem,2)
print ideal

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3.0
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -200.0
            Iterations: 5
            Function evaluations: 5
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9472.13571906
            Iterations: 3
            Function evaluations: 3
            Gradient evaluations: 2
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 150.218816494
            Iterations: 2
            Function evaluations: 13
            Gradient evaluations: 2
[3.0, -200.0, 9472.1357190646031]


## A posteriori methods

* A posteriori methods generate a representation of the Pareto optimal solutions, or the complete set of Pareto optimal solutions
* Benefits
  * The solutions can be visualized for problems with 2 or 3 objectives so the decision making is possible
  * When succesful, they give an understanding of the Pareto front
* Drawbacks
  * Approximating the Pareto optimal set often time-consuming
  * Decision making from a large representation may be very difficut


### The weighting method

Based on solving optimization problem
$$
\min_{x\in S} \sum_{i=1}^kw_if_i(x)
$$
for different weights $w_i\geq0$, $i=1,\ldots,k$. The idea is to generate weights evenly and then have evenly spread solutions.

In [119]:
import numpy as np
def weighting_method(f,w):
    points = []
    bounds = ((3,10),(50,200),(1000,4000)) #Bounds of the problem
    for wi in w:
        res=minimize(
            #weighted sum
            lambda x: sum(np.array(wi)*np.array(f(x[0],x[1],x[2]))), 
            [3,50,1000], method='SLSQP'
            #Jacobian using automatic differentiation
            ,jac=ad.gh(lambda x: sum(np.array(wi)*np.array(f(x[0],x[1],x[2]))))[0]
            #bounds given above
            ,bounds = bounds,options = {'disp':False})
        points.append((res.x,f(res.x[0],res.x[1],res.x[2])))
    return points

In [120]:
w = np.random.random((100,3))
repr = weighting_method(car_problem,w)
#print repr

In [121]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter([x[1][0] for x in repr],[x[1][1] for x in repr],[x[1][2] for x in repr])
plt.show()

### Epsilon-constraint method



In [24]:
import numpy as np
from scipy.optimize import minimize
import ad
def e_constraint_method(f,eps):
    points = []
    for epsi in eps:
        bounds = ((3,epsi[0]),(-epsi[1],200),(1000,4000)) #Added bounds for two first objectives
        res=minimize(
            #weighted sum
            lambda x: f(x[0],x[1],x[2])[2], 
            [3,50,1000], method='SLSQP'
            #Jacobian using automatic differentiation
            ,jac=ad.gh(lambda x: f(x[0],x[1],x[2])[2])[0]
            #bounds given above
            ,bounds = bounds,options = {'disp':False})
        if res.success:
            points.append((res.x,f(res.x[0],res.x[1],res.x[2])))
    return points

In [25]:
eps = np.random.random((100,2))
eps = [[epsi[0]*10,epsi[1]*(-200)] for epsi in eps]
print eps
repr = e_constraint_method(car_problem,eps)

[[9.3509371352646937, -83.544002286039557], [6.0768883797431181, -60.22922156045307], [2.6734555804184987, -159.6296919157285], [6.9133752631954852, -95.428842882044876], [6.1878502828003059, -146.86455831610695], [2.5970308859462667, -88.195703431900768], [0.50126476601471182, -46.979216004180337], [4.5282090945179583, -107.22767204367733], [6.7312059458405349, -112.6995506948855], [0.97522880712662352, -37.929457114215758], [2.5296705963282697, -61.442231011650541], [7.2441443180638654, -137.58717807651152], [0.89230937381892472, -47.392246342593822], [7.4513614959215477, -36.911195736784272], [5.2709445693020793, -34.61158173579377], [9.5458788613464503, -7.2340543846374228], [8.8779441273572477, -29.172375504220049], [3.5260729159651127, -38.345802607639932], [7.3822040336176276, -82.468802748721217], [9.9194487268803382, -34.654190358833567], [8.6897047645229399, -151.51174743538579], [4.2223027283401438, -142.17885049735887], [3.4953430633373159, -125.59341860064231], [9.77368991

In [26]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter([x[1][0] for x in repr],[x[1][1] for x in repr],[x[1][2] for x in repr])
plt.show()